In [ ]:
import pandas as pd
import numpy as np
import scipy.stats

# Load input files
module_genes_df = pd.read_excel("gtexv6_Substantianigra_darkorange2_4-3-2025.xlsx")
full_region_genes_df = pd.read_excel("CoExp_gtexv6_Substantianigra_GO-Report (4) (genes expressed in SN).xlsx")

# Extract gene list from darkorange2 module
module_genes = module_genes_df['name'].dropna().unique().tolist()

# Extract all genes expressed in the region from the GO enrichment report
gene_sets_column = full_region_genes_df['Unnamed: 7'].dropna()
all_genes_in_region = set()
for gene_string in gene_sets_column:
    genes = [g.strip() for g in gene_string.split(',')]
    all_genes_in_region.update(genes)
all_genes_in_region = list(all_genes_in_region)

# Load lipid genes from text file
lipid_genes = []
with open("lipid_genes.txt", "r") as f:
    for line in f:
        line = line.strip()
        stripped_line = line.split(',')
        for gene in stripped_line:
            lipid_genes.append(gene.strip())

# Chi-square test function
def chi_square(region_module_gene_list, full_region_gene_list, gene_of_interest_list, region_module_name, gene_list_name):
    region_no_module = set(full_region_gene_list).difference(set(region_module_gene_list))
    region_no_module = list(region_no_module)

    region_module_interest_match = sum(gene in gene_of_interest_list for gene in region_module_gene_list)
    region_no_module_interest_match = sum(gene in gene_of_interest_list for gene in region_no_module)

    df_dic = {
        gene_list_name: [region_module_interest_match, region_no_module_interest_match],
        f'not {gene_list_name}': [
            len(region_module_gene_list) - region_module_interest_match,
            len(region_no_module) - region_no_module_interest_match
        ],
        'total': [len(region_module_gene_list), len(region_no_module)]
    }

    observed_df = pd.DataFrame(df_dic).rename({0: region_module_name, 1: f'not {region_module_name}'})
    observed_df.loc['total'] = observed_df.sum()
    observed_df = observed_df.astype('float')

    expected_df = observed_df.iloc[0:2, 0:2].copy()
    total_n = observed_df.iloc[2, 2]
    for rowInd in range(2):
        for colInd in range(2):
            row_marg = observed_df.iloc[rowInd, -1]
            col_marg = observed_df.iloc[-1, colInd]
            expected_df.iloc[rowInd, colInd] = (row_marg * col_marg) / total_n

    chi_df = expected_df.copy()
    for rowInd in range(2):
        for colInd in range(2):
            o = observed_df.iloc[rowInd, colInd]
            e = expected_df.iloc[rowInd, colInd]
            chi_df.iloc[rowInd, colInd] = (o - e) ** 2 / e

    chi_stat = chi_df.to_numpy().sum()
    return observed_df, expected_df, chi_df, chi_stat

results = chi_square(
    region_module_gene_list=module_genes,
    full_region_gene_list=all_genes_in_region,
    gene_of_interest_list=lipid_genes,
    region_module_name='Substantianigra_darkorange2',
    gene_list_name='Lipid genes'
)

# Calculate p-value
pval = scipy.stats.chi2.sf(results[3], 1)

print("Observed:")
print(results[0])
print("\nExpected:")
print(results[1])
print("\nChi-Square Contributions:")
print(results[2])
print(f"\nChi-square Statistic: {results[3]:.4f}")
print(f"P-value: {pval:.4e}")





Observed:
                                 Lipid genes  not Lipid genes    total
Substantianigra_darkorange2             21.0            632.0    653.0
not Substantianigra_darkorange2        419.0          10766.0  11185.0
total                                  440.0          11398.0  11838.0

Expected:
                                 Lipid genes  not Lipid genes
Substantianigra_darkorange2        24.270992       628.729008
not Substantianigra_darkorange2   415.729008     10769.270992

Chi-Square Contributions:
                                 Lipid genes  not Lipid genes
Substantianigra_darkorange2         0.440830         0.017017
not Substantianigra_darkorange2     0.025736         0.000994

Chi-square Statistic: 0.4846
P-value: 4.8636e-01
